In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment1/q6_7.xlsx')
df = df.replace({'diagnosis': {'B': 0, 'M': 1}})
df.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [7]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1, random_state = 7).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

210 7


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,18.94,16.49,0.8750,6.445,3.639,5.0640,6.362,2
1,12.88,13.50,0.8879,5.139,3.119,2.3520,4.607,1
2,18.36,16.52,0.8452,6.666,3.485,4.9330,6.448,2
3,15.88,14.90,0.8988,5.618,3.507,0.7651,5.091,1
4,13.22,13.84,0.8680,5.395,3.070,4.1570,5.088,1


In [8]:
datan = df.values
X = datan[:, 0:7]
#X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
X = (X - X.mean(axis = 0))/(X.std(axis = 0))
#X = np.append(np.ones([m,1]), X, axis = 1)

y = datan[:, -1]# M -> 1, B -> 0  

w = np.zeros(n)
print(X.shape, y.shape)

(210, 7) (210,)


In [9]:
num_classes = len(np.unique(y))
print(num_classes)

3


In [3]:
def sigmoid(X, w):
  z = np.dot(X, w.T)
  return 1/(1+np.exp(-z.astype(float)))

In [10]:
def cost_L1(X, y, w, lm):
  m = len(y)
  h = sigmoid(X, w)
  return (-1/m)*np.sum(np.multiply(y, np.log(h))+ np.multiply(1-y, np.log(1-h))) + lm*np.sum(abs(w))

def cost_L2(X, y, w, lm):
  m = len(y)
  h = sigmoid(X, w)
  return (-1/m)*np.sum(np.multiply(y, np.log(h))+ np.multiply(1-y, np.log(1-h))) + (lm/(2))*np.sum((w)**2)

In [11]:
def bgd_L2(X, y, w, alpha, T, lm):
  costs = np.zeros(T)
  weights = np.zeros((T,7))
  m = len(y)
  for i in range(T):
    h = sigmoid(X, w)
    w = w*(1-alpha*lm) - (alpha/m)*np.dot(h-y,X)
    costs[i] = cost_L2(X, y, w, lm)
    weights[i, :] = w
  return costs, weights, w
################################################
def sgd_L2(X, y, w, alpha, T, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,7))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+1]
        ind_y = y[rand_index:rand_index+1]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w*(1-alpha*lm) - alpha * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L2(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w
################################################
def mbgd_L2(X, y, w, alpha, T, batch_size, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,7))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+batch_size]
        ind_y = y[rand_index:rand_index+batch_size]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w*(1-alpha*lm) - (alpha/batch_size) * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L2(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w

In [12]:
def bgd_L1(X, y, w, alpha, T, lm):
  costs = np.zeros(T)
  weights = np.zeros((T,7))
  m = len(y)
  for i in range(T):
    h = sigmoid(X, w)
    w = w - (alpha*lm/2)*(np.sign(w)) - (alpha/m)*np.dot(h-y,X)
    costs[i] = cost_L1(X, y, w, lm)
    weights[i, :] = w
  return costs, weights, w
################################################
def sgd_L1(X, y, w, alpha, T, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,7))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+1]
        ind_y = y[rand_index:rand_index+1]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w - (alpha*lm/2)*(np.sign(w)) - alpha * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L1(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w
################################################
def mbgd_L1(X, y, w, alpha, T, batch_size, lm):
    costs = np.zeros(T)
    weights = np.zeros((T,7))
    m = len(y)
    for i in range(T):
        rand_index = np.random.randint(m)
        ind_x = X[rand_index:rand_index+batch_size]
        ind_y = y[rand_index:rand_index+batch_size]
        z = ind_x.dot(w)
        h = 1/(1+np.exp(-z.astype(float)))
        w = w - (alpha*lm/2)*(np.sign(w)) - (alpha/batch_size) * (ind_x.T.dot(h - ind_y))
        costs[i] = cost_L1(ind_x,ind_y,w, lm)
        weights[i, :] = w
    return costs, weights, w

In [13]:
def grid_search_bgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  alpha_vals
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_bgd_L1, weights_bgd_L1, w_bgd_L1 = bgd_L1(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_bgd_L1))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_sgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_sgd_L1, weights_sgd_L1, w_sgd_L1 = sgd_L1(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_sgd_L1))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_mbgd_L1(X_train, X_val, y_train, y_val, w, alpha_vals, T, batch_size, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_mbgd_L1, weights_mbgd_L1, w_mbgd_L1 = mbgd_L1(X_train, y_train, w, i, T, batch_size, j)
      predictions = np.round_(sigmoid(X_val, w_mbgd_L1))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

In [15]:
def grid_search_bgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  acc_max = 0
  alpha_best = 0
  lm_best = 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_bgd_L2, weights_bgd_L2, w_bgd_L2 = bgd_L2(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_bgd_L2))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_sgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, T, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_sgd_L2, weights_sgd_L2, w_sgd_L2 = sgd_L2(X_train, y_train, w, i, T, j)
      predictions = np.round_(sigmoid(X_val, w_sgd_L2))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

def grid_search_mbgd_L2(X_train, X_val, y_train, y_val, w, alpha_vals, T, batch_size, lm_vals):
  acc_max = 0
  alpha_best, lm_best = 0, 0
  for i in alpha_vals:
    for j in lm_vals:
      costs_mbgd_L2, weights_mbgd_L2, w_mbgd_L2 = mbgd_L2(X_train, y_train, w, i, T, batch_size, j)
      predictions = np.round_(sigmoid(X_val, w_mbgd_L2))
      TN, TP, FN, FP = 0, 0, 0, 0
      for k in range(len(y_val)):
        if (predictions[k] == 0 and y_val[k] == 0):
          TN = TN + 1
        elif (predictions[k] == 1 and y_val[k] == 1):
          TP = TP + 1
        elif (predictions[k] == 0 and y_val[k] == 1):
          FN = FN + 1
        elif (predictions[k] == 1 and y_val[k] == 0):
          FP = FP + 1
      acc = (TP + TN)/(TP + TN + FN + FP)

      if(acc > acc_max):
        acc_max = acc
        alpha_best = i
        lm_best = j
  return alpha_best, lm_best

In [16]:
def confusion_matrix(predictions, labels):
  conf_matrix = [0,0,0,0,0,0,0,0,0]
  m = labels.shape[0]
  for i in range(m):
    conf_matrix[3*(labels[i]-1) + predictions[i] - 1] = conf_matrix[3*(labels[i]-1) + predictions[i] - 1] + 1 
  return conf_matrix

One vs All

In [17]:
alpha_vals = np.linspace(0.001, 0.01,20)
lm_vals = np.linspace(0.001,0.01,10)

iterations = 500

In [19]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  y_tr1 = (y_train == 1)*1
  y_te1 = (y_test == 1)*1
  y_va1 = (y_val == 1)*1

  y_tr2 = (y_train == 2)*2
  y_te2 = (y_test == 2)*2
  y_va2 = (y_val == 2)*2

  y_tr3 = (y_train == 3)*3
  y_te3 = (y_test == 3)*3
  y_va3 = (y_val == 3)*3

  w = np.zeros(n)
  alpha_best_bgd_L1_1, lm_best_bgd_L1_1 = grid_search_bgd_L1(X_train, X_val, y_tr1, y_va1, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L1_1, weights_bgd_L1_1, w_bgd_L1_1 = bgd_L1(X_train, y_tr1, w, alpha_best_bgd_L1_1, iterations, lm_best_bgd_L1_1)
  w = np.zeros(n)
  alpha_best_bgd_L1_2, lm_best_bgd_L1_2 = grid_search_bgd_L1(X_train, X_val, y_tr2, y_va2, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L1_2, weights_bgd_L1_2, w_bgd_L1_2 = bgd_L1(X_train, y_tr2, w, alpha_best_bgd_L1_2, iterations, lm_best_bgd_L1_2)
  w = np.zeros(n)
  alpha_best_bgd_L1_3, lm_best_bgd_L1_3 = grid_search_bgd_L1(X_train, X_val, y_tr3, y_va3, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L1_3, weights_bgd_L1_3, w_bgd_L1_3 = bgd_L1(X_train, y_tr3, w, alpha_best_bgd_L1_3, iterations, lm_best_bgd_L1_3)


  pred_bgd_L1_1 = (sigmoid(X_test, w_bgd_L1_1))
  pred_bgd_L1_2 = (sigmoid(X_test, w_bgd_L1_2))
  pred_bgd_L1_3 = (sigmoid(X_test, w_bgd_L1_3))

  y_pred = np.zeros(y_test.shape[0]).T
  for i in range(y_test.shape[0]):
    y_pred[i] = np.argmax([pred_bgd_L1_1[i],pred_bgd_L1_2[i],pred_bgd_L1_3[i]])+1
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.4931410256410269
Individual accuracy 2:  0.9800000000000011
Individual accuracy 3:  1.0
Overall accuracy :  0.8238095238095217


In [20]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  y_tr1 = (y_train == 1)*1
  y_te1 = (y_test == 1)*1
  y_va1 = (y_val == 1)*1

  y_tr2 = (y_train == 2)*2
  y_te2 = (y_test == 2)*2
  y_va2 = (y_val == 2)*2

  y_tr3 = (y_train == 3)*3
  y_te3 = (y_test == 3)*3
  y_va3 = (y_val == 3)*3

  w = np.zeros(n)
  alpha_best_bgd_L2_1, lm_best_bgd_L2_1 = grid_search_bgd_L2(X_train, X_val, y_tr1, y_va1, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L2_1, weights_bgd_L2_1, w_bgd_L2_1 = bgd_L2(X_train, y_tr1, w, alpha_best_bgd_L2_1, iterations, lm_best_bgd_L2_1)
  w = np.zeros(n)
  alpha_best_bgd_L2_2, lm_best_bgd_L2_2 = grid_search_bgd_L2(X_train, X_val, y_tr2, y_va2, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L2_2, weights_bgd_L2_2, w_bgd_L2_2 = bgd_L2(X_train, y_tr2, w, alpha_best_bgd_L2_2, iterations, lm_best_bgd_L2_2)
  w = np.zeros(n)
  alpha_best_bgd_L2_3, lm_best_bgd_L2_3 = grid_search_bgd_L2(X_train, X_val, y_tr3, y_va3, w, alpha_vals, iterations, lm_vals)
  costs_bgd_L2_3, weights_bgd_L2_3, w_bgd_L2_3 = bgd_L2(X_train, y_tr3, w, alpha_best_bgd_L2_3, iterations, lm_best_bgd_L2_3)


  pred_bgd_L2_1 = (sigmoid(X_test, w_bgd_L2_1))
  pred_bgd_L2_2 = (sigmoid(X_test, w_bgd_L2_2))
  pred_bgd_L2_3 = (sigmoid(X_test, w_bgd_L2_3))

  y_pred = np.zeros(y_test.shape[0]).T
  for i in range(y_test.shape[0]):
    y_pred[i] = np.argmax([pred_bgd_L2_1[i],pred_bgd_L2_2[i],pred_bgd_L2_3[i]])+1
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.5544230769230777
Individual accuracy 2:  0.9657142857142844
Individual accuracy 3:  0.9833333333333314
Overall accuracy :  0.8380952380952373


In [21]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  y_tr1 = (y_train == 1)*1
  y_te1 = (y_test == 1)*1
  y_va1 = (y_val == 1)*1

  y_tr2 = (y_train == 2)*2
  y_te2 = (y_test == 2)*2
  y_va2 = (y_val == 2)*2

  y_tr3 = (y_train == 3)*3
  y_te3 = (y_test == 3)*3
  y_va3 = (y_val == 3)*3

  w = np.zeros(n)
  alpha_best_sgd_L1_1, lm_best_sgd_L1_1 = grid_search_sgd_L1(X_train, X_val, y_tr1, y_va1, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L1_1, weights_sgd_L1_1, w_sgd_L1_1 = sgd_L1(X_train, y_tr1, w, alpha_best_sgd_L1_1, iterations, lm_best_sgd_L1_1)
  w = np.zeros(n)
  alpha_best_sgd_L1_2, lm_best_sgd_L1_2 = grid_search_sgd_L1(X_train, X_val, y_tr2, y_va2, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L1_2, weights_sgd_L1_2, w_sgd_L1_2 = sgd_L1(X_train, y_tr2, w, alpha_best_sgd_L1_2, iterations, lm_best_sgd_L1_2)
  w = np.zeros(n)
  alpha_best_sgd_L1_3, lm_best_sgd_L1_3 = grid_search_sgd_L1(X_train, X_val, y_tr3, y_va3, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L1_3, weights_sgd_L1_3, w_sgd_L1_3 = sgd_L1(X_train, y_tr3, w, alpha_best_sgd_L1_3, iterations, lm_best_sgd_L1_3)


  pred_sgd_L1_1 = (sigmoid(X_test, w_sgd_L1_1))
  pred_sgd_L1_2 = (sigmoid(X_test, w_sgd_L1_2))
  pred_sgd_L1_3 = (sigmoid(X_test, w_sgd_L1_3))

  y_pred = np.zeros(y_test.shape[0]).T
  for i in range(y_test.shape[0]):
    y_pred[i] = np.argmax([pred_sgd_L1_1[i],pred_sgd_L1_2[i],pred_sgd_L1_3[i]])+1
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.5390384615384617
Individual accuracy 2:  0.9857142857142833
Individual accuracy 3:  1.0
Overall accuracy :  0.8428571428571439


In [22]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  y_tr1 = (y_train == 1)*1
  y_te1 = (y_test == 1)*1
  y_va1 = (y_val == 1)*1

  y_tr2 = (y_train == 2)*2
  y_te2 = (y_test == 2)*2
  y_va2 = (y_val == 2)*2

  y_tr3 = (y_train == 3)*3
  y_te3 = (y_test == 3)*3
  y_va3 = (y_val == 3)*3

  w = np.zeros(n)
  alpha_best_sgd_L2_1, lm_best_sgd_L2_1 = grid_search_sgd_L2(X_train, X_val, y_tr1, y_va1, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L2_1, weights_sgd_L2_1, w_sgd_L2_1 = sgd_L2(X_train, y_tr1, w, alpha_best_sgd_L2_1, iterations, lm_best_sgd_L2_1)
  w = np.zeros(n)
  alpha_best_sgd_L2_2, lm_best_sgd_L2_2 = grid_search_sgd_L2(X_train, X_val, y_tr2, y_va2, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L2_2, weights_sgd_L2_2, w_sgd_L2_2 = sgd_L2(X_train, y_tr2, w, alpha_best_sgd_L2_2, iterations, lm_best_sgd_L2_2)
  w = np.zeros(n)
  alpha_best_sgd_L2_3, lm_best_sgd_L2_3 = grid_search_sgd_L2(X_train, X_val, y_tr3, y_va3, w, alpha_vals, iterations, lm_vals)
  costs_sgd_L2_3, weights_sgd_L2_3, w_sgd_L2_3 = sgd_L2(X_train, y_tr3, w, alpha_best_sgd_L2_3, iterations, lm_best_sgd_L2_3)


  pred_sgd_L2_1 = (sigmoid(X_test, w_sgd_L2_1))
  pred_sgd_L2_2 = (sigmoid(X_test, w_sgd_L2_2))
  pred_sgd_L2_3 = (sigmoid(X_test, w_sgd_L2_3))

  y_pred = np.zeros(y_test.shape[0]).T
  for i in range(y_test.shape[0]):
    y_pred[i] = np.argmax([pred_sgd_L2_1[i],pred_sgd_L2_2[i],pred_sgd_L2_3[i]])+1
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.46275641025641057
Individual accuracy 2:  1.0
Individual accuracy 3:  1.0
Overall accuracy :  0.8190476190476184


In [23]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  y_tr1 = (y_train == 1)*1
  y_te1 = (y_test == 1)*1
  y_va1 = (y_val == 1)*1

  y_tr2 = (y_train == 2)*2
  y_te2 = (y_test == 2)*2
  y_va2 = (y_val == 2)*2

  y_tr3 = (y_train == 3)*3
  y_te3 = (y_test == 3)*3
  y_va3 = (y_val == 3)*3

  w = np.zeros(n)
  alpha_best_mbgd_L1_1, lm_best_mbgd_L1_1 = grid_search_mbgd_L1(X_train, X_val, y_tr1, y_va1, w, alpha_vals, iterations,50, lm_vals)
  costs_mbgd_L1_1, weights_mbgd_L1_1, w_mbgd_L1_1 = mbgd_L1(X_train, y_tr1, w, alpha_best_mbgd_L1_1, iterations, 50, lm_best_mbgd_L1_1)
  w = np.zeros(n)
  alpha_best_mbgd_L1_2, lm_best_mbgd_L1_2 = grid_search_mbgd_L1(X_train, X_val, y_tr2, y_va2, w, alpha_vals, iterations,50, lm_vals)
  costs_mbgd_L1_2, weights_mbgd_L1_2, w_mbgd_L1_2 = mbgd_L1(X_train, y_tr2, w, alpha_best_mbgd_L1_2, iterations,50, lm_best_mbgd_L1_2)
  w = np.zeros(n)
  alpha_best_mbgd_L1_3, lm_best_mbgd_L1_3 = grid_search_mbgd_L1(X_train, X_val, y_tr3, y_va3, w, alpha_vals, iterations,50, lm_vals)
  costs_mbgd_L1_3, weights_mbgd_L1_3, w_mbgd_L1_3 = mbgd_L1(X_train, y_tr3, w, alpha_best_mbgd_L1_3, iterations,50, lm_best_mbgd_L1_3)


  pred_mbgd_L1_1 = (sigmoid(X_test, w_mbgd_L1_1))
  pred_mbgd_L1_2 = (sigmoid(X_test, w_mbgd_L1_2))
  pred_mbgd_L1_3 = (sigmoid(X_test, w_mbgd_L1_3))

  y_pred = np.zeros(y_test.shape[0]).T
  for i in range(y_test.shape[0]):
    y_pred[i] = np.argmax([pred_mbgd_L1_1[i],pred_mbgd_L1_2[i],pred_mbgd_L1_3[i]])+1
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.51147435897436
Individual accuracy 2:  0.9800000000000011
Individual accuracy 3:  1.0
Overall accuracy :  0.8285714285714265


In [25]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  y_tr1 = (y_train == 1)*1
  y_te1 = (y_test == 1)*1
  y_va1 = (y_val == 1)*1

  y_tr2 = (y_train == 2)*2
  y_te2 = (y_test == 2)*2
  y_va2 = (y_val == 2)*2

  y_tr3 = (y_train == 3)*3
  y_te3 = (y_test == 3)*3
  y_va3 = (y_val == 3)*3

  w = np.zeros(n)
  alpha_best_mbgd_L2_1, lm_best_mbgd_L2_1 = grid_search_mbgd_L2(X_train, X_val, y_tr1, y_va1, w, alpha_vals, iterations,50, lm_vals)
  costs_mbgd_L2_1, weights_mbgd_L2_1, w_mbgd_L2_1 = mbgd_L2(X_train, y_tr1, w, alpha_best_mbgd_L2_1, iterations,50, lm_best_mbgd_L2_1)
  w = np.zeros(n)
  alpha_best_mbgd_L2_2, lm_best_mbgd_L2_2 = grid_search_mbgd_L2(X_train, X_val, y_tr2, y_va2, w, alpha_vals, iterations,50, lm_vals)
  costs_mbgd_L2_2, weights_mbgd_L2_2, w_mbgd_L2_2 = mbgd_L2(X_train, y_tr2, w, alpha_best_mbgd_L2_2, iterations,50, lm_best_mbgd_L2_2)
  w = np.zeros(n)
  alpha_best_mbgd_L2_3, lm_best_mbgd_L2_3 = grid_search_mbgd_L2(X_train, X_val, y_tr3, y_va3, w, alpha_vals, iterations,50, lm_vals)
  costs_mbgd_L2_3, weights_mbgd_L2_3, w_mbgd_L2_3 = mbgd_L2(X_train, y_tr3, w, alpha_best_mbgd_L2_3, iterations,50, lm_best_mbgd_L2_3)


  pred_mbgd_L2_1 = (sigmoid(X_test, w_mbgd_L2_1))
  pred_mbgd_L2_2 = (sigmoid(X_test, w_mbgd_L2_2))
  pred_mbgd_L2_3 = (sigmoid(X_test, w_mbgd_L2_3))

  y_pred = np.zeros(y_test.shape[0]).T
  for i in range(y_test.shape[0]):
    y_pred[i] = np.argmax([pred_mbgd_L2_1[i],pred_mbgd_L2_2[i],pred_mbgd_L2_3[i]])+1
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.5842307692307683
Individual accuracy 2:  0.9800000000000011
Individual accuracy 3:  0.9894736842105261
Overall accuracy :  0.8476190476190428


One vs One


BGD + L1

In [26]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  X_train1 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==1:
      X_train1.append(X_train[i])
  X_train1 = np.array(X_train1)

  X_train2 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==2:
      X_train2.append(X_train[i])
  X_train2 = np.array(X_train2)

  X_train3 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==3:
      X_train3.append(X_train[i])
  X_train3 = np.array(X_train3)


  X_val1 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==1:
      X_val1.append(X_val[i])
  X_val1 = np.array(X_val1)

  X_val2 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==2:
      X_val2.append(X_val[i])
  X_val2 = np.array(X_val2)

  X_val3 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==3:
      X_val3.append(X_val[i])
  X_val3 = np.array(X_val3)

  #1v2
  T = 500
  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train2))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train2.shape[0])))

  XX_val = np.concatenate((X_val1,X_val2))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val2.shape[0])))

  alpha, lm = grid_search_bgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = bgd_L1(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v2 = np.round_(sigmoid(X_test,w))

  #2v3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train2,X_train3))
  yy_train = np.concatenate((np.ones(X_train2.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val2,X_val3))
  yy_val = np.concatenate((np.ones(X_val2.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_bgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = bgd_L1(XX_train, yy_train, w, alpha, T, lm)

  y_pred2v3 = np.round_(sigmoid(X_test,w))

  #1 vs 3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train3))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val1,X_val3))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_bgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = bgd_L1(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v3 = np.round_(sigmoid(X_test,w))

  for i in range(y_test.shape[0]):
    if y_pred1v2[i]==1:
      y_pred1v2[i]=1
    else:
      y_pred1v2[i]=2

    if y_pred2v3[i]==1:
      y_pred2v3[i]=2
    else:
      y_pred2v3[i]=3
    
    if y_pred1v3[i]==1:
      y_pred1v3[i]=1
    else:
      y_pred1v3[i]=3

  ls=[]
  for i in range(y_test.shape[0]):
    ls.append([y_pred1v2[i],y_pred2v3[i],y_pred1v3[i]])

  ls=np.array(ls)
  y_pred = np.zeros(y_test.shape[0]).T
  ls=ls.astype(int)
  for i in range(ls.shape[0]):
    y_pred[i]=(np.bincount(ls[i]).argmax())
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.6330769230769228
Individual accuracy 2:  0.9800000000000011
Individual accuracy 3:  0.9651515151515128
Overall accuracy :  0.8619047619047634


In [27]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  X_train1 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==1:
      X_train1.append(X_train[i])
  X_train1 = np.array(X_train1)

  X_train2 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==2:
      X_train2.append(X_train[i])
  X_train2 = np.array(X_train2)

  X_train3 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==3:
      X_train3.append(X_train[i])
  X_train3 = np.array(X_train3)


  X_val1 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==1:
      X_val1.append(X_val[i])
  X_val1 = np.array(X_val1)

  X_val2 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==2:
      X_val2.append(X_val[i])
  X_val2 = np.array(X_val2)

  X_val3 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==3:
      X_val3.append(X_val[i])
  X_val3 = np.array(X_val3)

  #1v2
  T = 500
  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train2))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train2.shape[0])))

  XX_val = np.concatenate((X_val1,X_val2))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val2.shape[0])))

  alpha, lm = grid_search_bgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = bgd_L2(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v2 = np.round_(sigmoid(X_test,w))

  #2v3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train2,X_train3))
  yy_train = np.concatenate((np.ones(X_train2.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val2,X_val3))
  yy_val = np.concatenate((np.ones(X_val2.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_bgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = bgd_L2(XX_train, yy_train, w, alpha, T, lm)

  y_pred2v3 = np.round_(sigmoid(X_test,w))

  #1 vs 3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train3))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val1,X_val3))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_bgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = bgd_L2(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v3 = np.round_(sigmoid(X_test,w))

  for i in range(y_test.shape[0]):
    if y_pred1v2[i]==1:
      y_pred1v2[i]=1
    else:
      y_pred1v2[i]=2

    if y_pred2v3[i]==1:
      y_pred2v3[i]=2
    else:
      y_pred2v3[i]=3
    
    if y_pred1v3[i]==1:
      y_pred1v3[i]=1
    else:
      y_pred1v3[i]=3

  ls=[]
  for i in range(y_test.shape[0]):
    ls.append([y_pred1v2[i],y_pred2v3[i],y_pred1v3[i]])

  ls=np.array(ls)
  y_pred = np.zeros(y_test.shape[0]).T
  ls=ls.astype(int)
  for i in range(ls.shape[0]):
    y_pred[i]=(np.bincount(ls[i]).argmax())
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.6064102564102561
Individual accuracy 2:  0.9800000000000011
Individual accuracy 3:  0.9651515151515128
Overall accuracy :  0.8523809523809541


In [28]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  X_train1 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==1:
      X_train1.append(X_train[i])
  X_train1 = np.array(X_train1)

  X_train2 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==2:
      X_train2.append(X_train[i])
  X_train2 = np.array(X_train2)

  X_train3 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==3:
      X_train3.append(X_train[i])
  X_train3 = np.array(X_train3)


  X_val1 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==1:
      X_val1.append(X_val[i])
  X_val1 = np.array(X_val1)

  X_val2 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==2:
      X_val2.append(X_val[i])
  X_val2 = np.array(X_val2)

  X_val3 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==3:
      X_val3.append(X_val[i])
  X_val3 = np.array(X_val3)

  #1v2
  T = 500
  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train2))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train2.shape[0])))

  XX_val = np.concatenate((X_val1,X_val2))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val2.shape[0])))

  alpha, lm = grid_search_sgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = sgd_L1(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v2 = np.round_(sigmoid(X_test,w))

  #2v3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train2,X_train3))
  yy_train = np.concatenate((np.ones(X_train2.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val2,X_val3))
  yy_val = np.concatenate((np.ones(X_val2.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_sgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = sgd_L1(XX_train, yy_train, w, alpha, T, lm)

  y_pred2v3 = np.round_(sigmoid(X_test,w))

  #1 vs 3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train3))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val1,X_val3))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_sgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = sgd_L1(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v3 = np.round_(sigmoid(X_test,w))

  for i in range(y_test.shape[0]):
    if y_pred1v2[i]==1:
      y_pred1v2[i]=1
    else:
      y_pred1v2[i]=2

    if y_pred2v3[i]==1:
      y_pred2v3[i]=2
    else:
      y_pred2v3[i]=3
    
    if y_pred1v3[i]==1:
      y_pred1v3[i]=1
    else:
      y_pred1v3[i]=3

  ls=[]
  for i in range(y_test.shape[0]):
    ls.append([y_pred1v2[i],y_pred2v3[i],y_pred1v3[i]])

  ls=np.array(ls)
  y_pred = np.zeros(y_test.shape[0]).T
  ls=ls.astype(int)
  for i in range(ls.shape[0]):
    y_pred[i]=(np.bincount(ls[i]).argmax())
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.594743589743589
Individual accuracy 2:  0.9800000000000011
Individual accuracy 3:  0.9818181818181813
Overall accuracy :  0.8523809523809552


In [29]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  X_train1 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==1:
      X_train1.append(X_train[i])
  X_train1 = np.array(X_train1)

  X_train2 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==2:
      X_train2.append(X_train[i])
  X_train2 = np.array(X_train2)

  X_train3 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==3:
      X_train3.append(X_train[i])
  X_train3 = np.array(X_train3)


  X_val1 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==1:
      X_val1.append(X_val[i])
  X_val1 = np.array(X_val1)

  X_val2 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==2:
      X_val2.append(X_val[i])
  X_val2 = np.array(X_val2)

  X_val3 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==3:
      X_val3.append(X_val[i])
  X_val3 = np.array(X_val3)

  #1v2
  T = 500
  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train2))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train2.shape[0])))

  XX_val = np.concatenate((X_val1,X_val2))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val2.shape[0])))

  alpha, lm = grid_search_sgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = sgd_L2(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v2 = np.round_(sigmoid(X_test,w))

  #2v3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train2,X_train3))
  yy_train = np.concatenate((np.ones(X_train2.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val2,X_val3))
  yy_val = np.concatenate((np.ones(X_val2.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_sgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = sgd_L2(XX_train, yy_train, w, alpha, T, lm)

  y_pred2v3 = np.round_(sigmoid(X_test,w))

  #1 vs 3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train3))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val1,X_val3))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_sgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, lm_vals)
  costs, weights, w = sgd_L2(XX_train, yy_train, w, alpha, T, lm)

  y_pred1v3 = np.round_(sigmoid(X_test,w))

  for i in range(y_test.shape[0]):
    if y_pred1v2[i]==1:
      y_pred1v2[i]=1
    else:
      y_pred1v2[i]=2

    if y_pred2v3[i]==1:
      y_pred2v3[i]=2
    else:
      y_pred2v3[i]=3
    
    if y_pred1v3[i]==1:
      y_pred1v3[i]=1
    else:
      y_pred1v3[i]=3

  ls=[]
  for i in range(y_test.shape[0]):
    ls.append([y_pred1v2[i],y_pred2v3[i],y_pred1v3[i]])

  ls=np.array(ls)
  y_pred = np.zeros(y_test.shape[0]).T
  ls=ls.astype(int)
  for i in range(ls.shape[0]):
    y_pred[i]=(np.bincount(ls[i]).argmax())
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.6043589743589729
Individual accuracy 2:  0.9800000000000011
Individual accuracy 3:  0.9651515151515128
Overall accuracy :  0.8523809523809535


In [30]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  X_train1 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==1:
      X_train1.append(X_train[i])
  X_train1 = np.array(X_train1)

  X_train2 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==2:
      X_train2.append(X_train[i])
  X_train2 = np.array(X_train2)

  X_train3 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==3:
      X_train3.append(X_train[i])
  X_train3 = np.array(X_train3)


  X_val1 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==1:
      X_val1.append(X_val[i])
  X_val1 = np.array(X_val1)

  X_val2 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==2:
      X_val2.append(X_val[i])
  X_val2 = np.array(X_val2)

  X_val3 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==3:
      X_val3.append(X_val[i])
  X_val3 = np.array(X_val3)

  #1v2
  T = 500
  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train2))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train2.shape[0])))

  XX_val = np.concatenate((X_val1,X_val2))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val2.shape[0])))

  alpha, lm = grid_search_mbgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, 50, lm_vals)
  costs, weights, w = mbgd_L1(XX_train, yy_train, w, alpha, T,50,  lm)

  y_pred1v2 = np.round_(sigmoid(X_test,w))

  #2v3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train2,X_train3))
  yy_train = np.concatenate((np.ones(X_train2.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val2,X_val3))
  yy_val = np.concatenate((np.ones(X_val2.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_mbgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T,50, lm_vals)
  costs, weights, w = mbgd_L1(XX_train, yy_train, w, alpha, T,50, lm)

  y_pred2v3 = np.round_(sigmoid(X_test,w))

  #1 vs 3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train3))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val1,X_val3))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_mbgd_L1(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T,50, lm_vals)
  costs, weights, w = mbgd_L1(XX_train, yy_train, w, alpha, T,50, lm)

  y_pred1v3 = np.round_(sigmoid(X_test,w))

  for i in range(y_test.shape[0]):
    if y_pred1v2[i]==1:
      y_pred1v2[i]=1
    else:
      y_pred1v2[i]=2

    if y_pred2v3[i]==1:
      y_pred2v3[i]=2
    else:
      y_pred2v3[i]=3
    
    if y_pred1v3[i]==1:
      y_pred1v3[i]=1
    else:
      y_pred1v3[i]=3

  ls=[]
  for i in range(y_test.shape[0]):
    ls.append([y_pred1v2[i],y_pred2v3[i],y_pred1v3[i]])

  ls=np.array(ls)
  y_pred = np.zeros(y_test.shape[0]).T
  ls=ls.astype(int)
  for i in range(ls.shape[0]):
    y_pred[i]=(np.bincount(ls[i]).argmax())
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.4269230769230772
Individual accuracy 2:  1.0
Individual accuracy 3:  1.0
Overall accuracy :  0.8047619047619031


In [31]:
kf = KFold(n_splits=5)
overall = []
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
for train_index, test_index in kf.split(X):
  X_train_val, X_test = X[train_index], X[test_index]
  y_train_val, y_test = y[train_index], y[test_index]
  X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)

  X_train1 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==1:
      X_train1.append(X_train[i])
  X_train1 = np.array(X_train1)

  X_train2 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==2:
      X_train2.append(X_train[i])
  X_train2 = np.array(X_train2)

  X_train3 = []
  for i in range(y_train.shape[0]):
    if y_train[i]==3:
      X_train3.append(X_train[i])
  X_train3 = np.array(X_train3)


  X_val1 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==1:
      X_val1.append(X_val[i])
  X_val1 = np.array(X_val1)

  X_val2 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==2:
      X_val2.append(X_val[i])
  X_val2 = np.array(X_val2)

  X_val3 = []
  for i in range(y_val.shape[0]):
    if y_val[i]==3:
      X_val3.append(X_val[i])
  X_val3 = np.array(X_val3)

  #1v2
  T = 500
  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train2))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train2.shape[0])))

  XX_val = np.concatenate((X_val1,X_val2))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val2.shape[0])))

  alpha, lm = grid_search_mbgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T, 50, lm_vals)
  costs, weights, w = mbgd_L2(XX_train, yy_train, w, alpha, T,50,  lm)

  y_pred1v2 = np.round_(sigmoid(X_test,w))

  #2v3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train2,X_train3))
  yy_train = np.concatenate((np.ones(X_train2.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val2,X_val3))
  yy_val = np.concatenate((np.ones(X_val2.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_mbgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T,50, lm_vals)
  costs, weights, w = mbgd_L2(XX_train, yy_train, w, alpha, T,50, lm)

  y_pred2v3 = np.round_(sigmoid(X_test,w))

  #1 vs 3

  w = np.zeros(7).T

  XX_train = np.concatenate((X_train1,X_train3))
  yy_train = np.concatenate((np.ones(X_train1.shape[0]), np.zeros(X_train3.shape[0])))

  XX_val = np.concatenate((X_val1,X_val3))
  yy_val = np.concatenate((np.ones(X_val1.shape[0]), np.zeros(X_val3.shape[0])))

  alpha, lm = grid_search_mbgd_L2(XX_train, XX_val, yy_train, yy_val, w, alpha_vals, T,50, lm_vals)
  costs, weights, w = mbgd_L2(XX_train, yy_train, w, alpha, T,50, lm)

  y_pred1v3 = np.round_(sigmoid(X_test,w))

  for i in range(y_test.shape[0]):
    if y_pred1v2[i]==1:
      y_pred1v2[i]=1
    else:
      y_pred1v2[i]=2

    if y_pred2v3[i]==1:
      y_pred2v3[i]=2
    else:
      y_pred2v3[i]=3
    
    if y_pred1v3[i]==1:
      y_pred1v3[i]=1
    else:
      y_pred1v3[i]=3

  ls=[]
  for i in range(y_test.shape[0]):
    ls.append([y_pred1v2[i],y_pred2v3[i],y_pred1v3[i]])

  ls=np.array(ls)
  y_pred = np.zeros(y_test.shape[0]).T
  ls=ls.astype(int)
  for i in range(ls.shape[0]):
    y_pred[i]=(np.bincount(ls[i]).argmax())
  for i in range(y_test.shape[0]):
    conf_matrix = confusion_matrix(y_pred.astype(int),y_test.astype(int))
    ind_acc1.append(conf_matrix[0]/sum(conf_matrix[0:3]))
    ind_acc2.append(conf_matrix[4]/sum(conf_matrix[3:6]))
    ind_acc3.append(conf_matrix[8]/sum(conf_matrix[6:9])) 
    overall.append((conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix))
print("Individual accuracy 1: ",sum(ind_acc1)/len(ind_acc1))
print("Individual accuracy 2: ",sum(ind_acc2)/len(ind_acc2))
print("Individual accuracy 3: ",sum(ind_acc3)/len(ind_acc3))
print("Overall accuracy : ", sum(overall)/len(overall))

Individual accuracy 1:  0.481858974358974
Individual accuracy 2:  1.0
Individual accuracy 3:  1.0
Overall accuracy :  0.8238095238095254
